In [ ]:
from datetime import datetime
from summer2.utils import Epoch
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

from aust_covid.inputs import load_household_impacts_data
from summer2.functions.time import get_linear_interpolation_function

In [ ]:
aust_epoch = Epoch(datetime(2019, 12, 31))

In [ ]:
hh_impact = load_household_impacts_data()
hh_test_ratio = hh_impact["test"] / hh_impact["sympts"]

In [ ]:
cdr_func = get_linear_interpolation_function(aust_epoch.dti_to_index(hh_test_ratio.index), hh_test_ratio.to_numpy())

In [ ]:
eval_start = aust_epoch.datetime_to_number(datetime(2022, 1, 1))
eval_end = aust_epoch.datetime_to_number(datetime(2022, 12, 31))
n_times = 40
eval_times = np.linspace(eval_start, eval_end, n_times)

In [ ]:
out_array = pd.Series(
    [float(cdr_func.evaluate(model_variables={"time": i})) for i in eval_times], 
    index=aust_epoch.index_to_dti(eval_times),
)
out_array.plot()

In [ ]:
def get_param(test_ratio, cdr):
    return -np.log(1.0 - cdr) / test_ratio

In [ ]:
starting_cdr = 0.1
param = get_param(hh_test_ratio[0], starting_cdr)
input_values = np.linspace(0.0, 5.0, 50)
cdr_values = 1.0 - np.exp(-input_values * param)
pd.Series(cdr_values, index=input_values).plot()